In [40]:
#just need to extract the time for the LED pulse and produce a CSV with this in.
#LED square pulse should be going up to 2.9V but triggered when 960mV - find time on oscilloscope when this is true and the index of this

In [41]:
import sys
import time
import matplotlib.pyplot as plt 
import numpy as np 
import tekwfm
import os
from os.path import exists
from scipy import signal
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.special import gamma
import matplotlib.cm as cm
from collections import Counter

import gc
import re
import tempfile
from collections import defaultdict, namedtuple
from pathlib import Path
import pandas as pd

In [42]:
doVerbose = False

#import the wfm file
filename_base = "091225_spe_2000_2500_1_on_ch2"

#saving the processed file

folder_name = "PMT1_SPE_CLEAN"
file_name = "091225_spe_2000_2500_1_on_ch2_processed.csv" 
file_path = os.path.join(os.getcwd(), folder_name, file_name)


startEvent = 0
nEvents = 2500


volts, tstart, tscale, tfrac, tdatefrac, tdate = tekwfm.read_wfm(filename_base+".wfm")
if doVerbose:
    print('>>>>', volts, tstart, tscale, tfrac, tdatefrac, tdate)

samples = volts.shape

print(len(volts))
print(tscale)
print('samples', samples[0])
tstop = samples[0]*tscale+tstart
sampleTimes = [tstart+x*tscale for x in range(samples[0])]

#already positive, don't need to flip like with CH1.
tempVolts = [volts[:, event] for event in range(startEvent, startEvent + nEvents)] # all events, array of array of voltages



12500
8e-11
samples 12500


In [43]:
def first_crossing_time(tempVolts, sampleTimes, threshold=0.960):

    for i in range(1, len(tempVolts)):
        if tempVolts[i-1] < threshold <= tempVolts[i]:
            #linear interpolation
            y1, y2 = tempVolts[i-1], tempVolts[i]
            x1, x2 = sampleTimes[i-1], sampleTimes[i]
    
            if (y2 - y1)==0:
                return None, None, None
            frac = (threshold - y1) / (y2 - y1)
            if not (0<=frac<=1):
                return None, None, None
        
            first_cross = x1 + frac * (x2 - x1)

            return first_cross
    
    #if no crossing found
    return None

def cross_index(first_cross, sampleTimes):
    if first_cross is None:
        return len(sampleTimes) - 1
    else:
        time_difference = np.abs(np.array(sampleTimes) - first_cross)
        cross_index = np.argmin(time_difference)
        return cross_index



In [44]:
#for all events
cross_times = []
cross_indices = []

for tempVolt in tempVolts:
    t_cross = first_crossing_time(tempVolt, sampleTimes)
    i_cross = cross_index(t_cross, sampleTimes)
    cross_times.append(t_cross)
    cross_indices.append(i_cross)

print(cross_times)
print(cross_indices)

[np.float64(-1.272033333333333e-07), np.float64(-2.9017599999999996e-07), np.float64(-1.440592592592592e-07), np.float64(-2.4855384615384613e-07), np.float64(-2.3737249999999998e-07), None, np.float64(-2.432463157894736e-07), np.float64(-2.430133333333333e-07), np.float64(-1.6663999999999996e-07), None, np.float64(-3.3168941176470585e-07), np.float64(-3.438117647058823e-07), np.float64(-2.1290857142857138e-07), None, np.float64(-2.138307692307692e-07), np.float64(-1.5120827586206895e-07), np.float64(1.6788965517241372e-08), np.float64(-2.673148717948718e-07), np.float64(-1.7253629629629627e-07), np.float64(-3.094333333333333e-07), np.float64(-6.053803921568624e-08), np.float64(-3.9654956521739127e-07), None, np.float64(-3.199875e-07), np.float64(-7.390769230769206e-09), np.float64(-1.8949999999999997e-07), None, np.float64(-2.905727272727272e-07), None, np.float64(-1.898153846153846e-08), np.float64(-2.493772549019607e-07), np.float64(-3.673264516129032e-07), np.float64(-3.632690909090

In [45]:
#produce the csv file

df = pd.DataFrame({
            'crossing_trigger': cross_times,
            'crossing_index': cross_indices,
            })

df.to_csv(file_path, sep=',', encoding='utf-8-sig', index=True, header=True)

print('CSV done!')

CSV done!
